In [3]:
from glob import glob
from astroquery.mast import Mast, Observations
import os
import shutil
import numpy as np
import json
import requests
import asdf
from astropy import log
from astropy.io import ascii, fits
from astropy.utils.data import download_file
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import Table

from align_to_catalogs import *
import astropy.units as u
from reproject.mosaicking import find_optimal_celestial_wcs

In [34]:
import jwst

print(jwst.__version__)
JP-3285: Allow negative angles in engineering format. Add support for angle units by @mcara in #7683


2023-08-03T16:50:14.662534: 1.11.1.dev1+g99753465


In [2]:
#realign_to_catalog(reference_coordinates, filtername='f212n',
#                       module='nrca',
#                       basepath='/orange/adamginsburg/jwst/brick/',
#                       fieldnumber='001',
#                       max_offset=0.4*u.arcsec,
#                       mag_limit=15,
#                       catfile=None, imfile=None)
#
#

In [3]:
0 + 1*u.arcsec

<Quantity 1. arcsec>

In [5]:
fov_regname = {'brick': 'regions/nircam_brick_fov.reg',
              'cloudc': 'regions/nircam_cloudc_fov.reg',
              }
regionname = 'cloudc'
basepath = f'/orange/adamginsburg/jwst/{regionname}/'
filtername = 'F405N'
field = '002' 
module = 'merged'
output_dir = f'/orange/adamginsburg/jwst/{regionname}/{filtername}/pipeline/'

In [5]:
imfile = '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_02201_00002_nrcalong_destreak.fits'
imfile_realigned = '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_02201_00002_nrcalong_destreak_realigned_to_vvv.fits'
shutil.copy(imfile, imfile_realigned)

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_02201_00002_nrcalong_destreak_realigned_to_vvv.fits'

In [6]:
reference_catalog = '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_t001_nircam_clear-F405N-merged_vvvcat.fits'

In [7]:
reftbl = Table.read(reference_catalog)

In [8]:
reference_coordinates = reftbl['skycoord']

In [9]:
reftbl['skycoord']

<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    [(266.581129, -28.666482), (266.594517, -28.666434),
     (266.604305, -28.663877), ..., (266.555803, -28.513283),
     (266.560494, -28.514834), (266.559398, -28.51401 )]>

In [10]:
cat_file = '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv'
cat = Table.read(cat_file)

In [15]:
#cat['xcentroid'], cat['ycentroid']

In [12]:
ww =  WCS(fits.getheader(imfile, ext=('SCI', 1)))
ww

Set DATE-AVG to '2023-04-25T04:14:32.839' from MJD-AVG.
Set DATE-END to '2023-04-25T04:14:59.681' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to   -11.485975 from OBSGEO-[XYZ].
Set OBSGEO-H to 1386082407.725 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN-SIP'  'DEC--TAN-SIP'  
CRVAL : 266.59237008461  -28.557172894513  
CRPIX : 1024.5  1024.5  
CD1_1 CD1_2  : -1.6126880967505e-06  -1.7440453913269e-05  
CD2_1 CD2_2  : -1.7358691201541e-05  1.6484453417233e-06  
NAXIS : 2048  2048

In [13]:
skycrds_cat = ww.pixel_to_world(cat['xcentroid'], cat['ycentroid'])
skycrds_cat

<SkyCoord (ICRS): (ra, dec) in deg
    [(266.6116505 , -28.56882209), (266.61113913, -28.57424765),
     (266.61090323, -28.57649018), ..., (266.54867726, -28.65584137),
     (266.54533313, -28.67274956), (266.54349879, -28.68148105)]>

In [16]:
ww.wcs.crval

array([266.59237008, -28.55717289])

In [17]:
#ww.wcs.crval = ww.wcs.crval - [(0.1*u.arcsec).to(u.deg).value, (0.1*u.arcsec).to(u.deg).value]
#ww.wcs.crval

In [18]:
#realign_to_catalog(reference_coordinates=reference_coordinates, filtername=filtername.lower(), module=module, 
#                   basepath=basepath, fieldnumber=field, imfile=imfile_realigned, decoffset=-8*u.arcsec)

In [19]:
from jwst.scripts import asn_edit

In [19]:
asn_edit??

Type:        module
String form: <module 'jwst.scripts.asn_edit' from '/blue/adamginsburg/adamginsburg/repos/jwst/jwst/scripts/asn_edit.py'>
File:        /blue/adamginsburg/adamginsburg/repos/jwst/jwst/scripts/asn_edit.py
Source:     
#!/usr/bin/env python

import argparse

from jwst.associations import asn_edit


def main():
    """
    Parse command line, read, edit, write association file
    """

    # Parse command line arguments
    description_text = """
Edit Association File

This script adds or removes filenames from an association file. The
first argument is the name of the association file. Subsequent
arguments are the filenames. Options determine which operation is
performed: --add or --remove. If adding files the --type option sets
the exposure type for the new files. If removing file, the --ignore
option will not use the filename suffix when matching the filenames for
removal. Normally the output association file is the same as the input.
The --output option allows you to

In [20]:
shutil.copy('/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230622t004213_image3_00004_nrca_asn.json', 
            '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230622t004213_image3_00004_nrca_asn_copy.json')

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230622t004213_image3_00004_nrca_asn_copy.json'

In [22]:
asn_edit('/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_20230622t004213_image3_00004_nrca_asn_copy.json')

TypeError: 'module' object is not callable

In [ ]:
#realign_to_vvv(filtername=filtername.lower(), fov_regname=fov_regname[regionname], basepath=basepath, module=module, fieldnumber=field,
#                                   imfile=realigned_vvv_filename, ksmag_limit=15 if filtername=='f410m' else 11, mag_limit=15)

In [3]:
ims = glob(os.path.join(output_dir, f'*nrca*_destreak.fits'))
ims

['/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00001_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00002_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_02201_00003_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_02201_00002_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00004_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00003_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00002_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00001_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_02201_00004_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00003_nrcalong_destre

In [6]:
i = ims[0]
i1 = i.split('.')[0]+'_realigned.fits'
i1

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00001_nrcalong_destreak_realigned.fits'

In [21]:
i

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_02201_00003_nrcalong_destreak.fits'

In [11]:
i1.split('_')[0][-3:]

'002'

In [18]:
i1

'/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00001_nrcalong_destreak_realigned.fits'

In [33]:
realigned_vvv_i = i.split('.')[0]+'_realigned.fits'
shutil.copy(i, realigned_vvv_i)

raoffset = -1.65*u.arcsec
decoffset = -3.9*u.arcsec

realign_to_vvv(filtername=filtername.lower(), fov_regname=fov_regname[regionname], basepath=basepath, module=module, fieldnumber=field, 
               imfile=realigned_vvv_i, ksmag_limit=15 if filtername=='f410m' else 11, mag_limit=15, raoffset=raoffset, decoffset=decoffset)

INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9430 of 19563 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-02T18:57:48.707773: Before realignment, offset is -0.054476696561778226 arcsec, 0.015985920926198105 arcsec.  Found 256 matches.
2023-08-02T18:57:48.715219: CRVAL before 266.59154983035 -28.556436704257
2023-08-02T18:57:48.750006: CRVAL after 266.5920232961 -28.555357811457
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
   

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x2b159089a670>, <astropy.io.fits.hdu.image.ImageHDU object at 0x2b159013d430>, <astropy.io.fits.hdu.image.ImageHDU object at 0x2b1599e75f10>, <astropy.io.fits.hdu.image.ImageHDU object at 0x2b15995112e0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x2b158f8e4700>, <astropy.io.fits.hdu.image.ImageHDU object at 0x2b158f8e4100>, <astropy.io.fits.hdu.image.ImageHDU object at 0x2b158f8e4be0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x2b158f8e4940>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x2b158f8e4160>]

In [16]:
for i in ims:
    realigned_vvv_i = i.split('.')[0]+'_realigned.fits'
    shutil.copy(i, realigned_vvv_i)
    visit = i.split('_')[0][-3:]
    if visit == '001':
        raoffset = 0*u.arcsec
        decoffset = -8*u.arcsec
    elif visit == '002':
        raoffset = -1*u.arcsec
        decoffset = -4*u.arcsec
    else:
        raoffset = 0*u.arcsec
        decoffset = 0*u.arcsec
    realign_to_vvv(filtername=filtername.lower(), fov_regname=fov_regname[regionname], basepath=basepath, module=module, fieldnumber=field, 
                   imfile=realigned_vvv_i, ksmag_limit=15 if filtername=='f410m' else 11, mag_limit=15, raoffset=raoffset, decoffset=decoffset)

INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9414 of 19532 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-01T13:37:08.264530: Before realignment, offset is -0.030882441615176504 arcsec, -0.0033957754496327652 arcsec.  Found 303 matches.
2023-08-01T13:37:08.273922: CRVAL before 266.59541974652 -28.538974425935
2023-08-01T13:37:08.287062: CRVAL after 266.59570610275 -28.537862371553
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,


2023-08-01T13:37:08.406572: After re-realignment, offset is -0.022481750784208998 arcsec, -0.004181148023008063 arcsec using 293 matches
INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9414 of 19532 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-01T13:37:10.757341: Before realignment, offset is -0.01951065955836384 arcsec, -0.013600705611338526 arcsec.  Found 221 matches.
2023-08-01T13:37:10.767009: CRVAL before 266.58849083407 -28.575359801581
2023-08-01T13:37:10.781919: CRVAL after 266.5884962537 -28.573133801385
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                theref

2023-08-01T13:37:11.105410: After realignment, offset is -0.012574651566410466 arcsec, -0.015839801412909083 arcsec with 222 mathces
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consist

INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9414 of 19532 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-01T13:37:15.853893: Before realignment, offset is -0.024809693115912523 arcsec, 0.01804609102151744 arcsec.  Found 256 matches.
2023-08-01T13:37:15.863249: CRVAL before 266.59237008461 -28.557172894513
2023-08-01T13:37:15.877087: CRVAL after 266.59265475397 -28.556066796205
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
   

2023-08-01T13:37:15.993452: After re-realignment, offset is -0.027299778207634517 arcsec, 0.013845916496535438 arcsec using 260 matches
INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9414 of 19532 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-01T13:37:18.623536: Before realignment, offset is -0.024903513440222014 arcsec, -0.05622041633515096 arcsec.  Found 184 matches.
2023-08-01T13:37:18.632759: CRVAL before 266.58873993895 -28.574669560401
2023-08-01T13:37:18.647462: CRVAL after 266.58874685659 -28.572431721396
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                theref

...

INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9414 of 19532 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-01T13:37:36.070103: Before realignment, offset is -0.040571443855697 arcsec, -0.03702422725524457 arcsec.  Found 273 matches.
2023-08-01T13:37:36.078616: CRVAL before 266.59261881688 -28.556482695308
2023-08-01T13:37:36.092077: CRVAL after 266.5929078645 -28.555361299689
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
      

2023-08-01T13:37:36.561083: After re-realignment, offset is -0.045185107137513114 arcsec, -0.03469746114745931 arcsec using 282 matches
INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9414 of 19532 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-01T13:37:38.883451: Before realignment, offset is -0.03825505508530114 arcsec, -0.01992610309073939 arcsec.  Found 186 matches.
2023-08-01T13:37:38.892891: CRVAL before 266.58460781321 -28.593550093747
2023-08-01T13:37:38.907294: CRVAL after 266.58461843961 -28.591322336496
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefo

2023-08-01T13:37:39.023554: After re-realignment, offset is -0.030904571599421615 arcsec, -0.004442837087026419 arcsec using 186 matches
INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9414 of 19532 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-01T13:37:41.267928: Before realignment, offset is -0.03074188020946167 arcsec, -0.0373162116247272 arcsec.  Found 219 matches.
2023-08-01T13:37:41.277796: CRVAL before 266.59154053673 -28.55716467436
2023-08-01T13:37:41.291061: CRVAL after 266.59182685392 -28.556043197635
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefor

2023-08-01T13:37:41.408912: After re-realignment, offset is -0.028291420176174142 arcsec, -0.03293204669603256 arcsec using 223 matches
INFO: Kept 7752 out of 9945 VVV stars using ksmag_limit>11 [align_to_catalogs]
INFO: For f405n merged 002 catalog /orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-f405n-merged_cat.ecsv, found 9414 of 19532 sources meeting criteria mag<15 [align_to_catalogs]
2023-08-01T13:37:43.704289: Before realignment, offset is -0.00997715246739972 arcsec, 0.010978262529448557 arcsec.  Found 200 matches.
2023-08-01T13:37:43.714477: CRVAL before 266.58377837707 -28.593542796145
2023-08-01T13:37:43.728552: CRVAL after 266.5837811485 -28.59132362344
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore

2023-08-01T13:37:44.071111: After realignment, offset is -0.01645709437525511 arcsec, 0.016203974610107252 arcsec with 195 mathces
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consisten